In [1]:
# import requests
import time
from pprint import pprint
import json
import csv
import os
from dotenv import load_dotenv, dotenv_values
load_dotenv()
from datetime import datetime, timedelta
import requests

# Set your start date

In [2]:
#Set start_data
#start_date = '2023-11-23'
start_date = '2024-09-09'

In [3]:
# Returns a string that is 30 days after the date given. So if input is a string that says 2024-04-18, returns a string that says 2024-05-18
def date_plus_thirty(date):
    
    dt_start = datetime.strptime(date, '%Y-%m-%d')
    dt_end = dt_start + timedelta(30)
    
    return dt_end.strftime('%Y-%m-%d')

print(date_plus_thirty('2024-04-18'))

2024-05-18


# Get Activity Log List

In [4]:
#Takes start_date, start of batch gathering
#Takes end_date, if None, then will default to today, if specified, then will use that date. 
#Returns of dates that are the edges of each 30 day interval
def get_date_intervals(start_date, end_date):
    #Variable initialization
    date_intervals = []
    end_date_dt = None
    
    #Initialize start_date_dt obj
    try:
        start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
    except:
        raise ValueError("Please provide dates in format of YYYY-MM-DD")
        
        
    #Initialize and set end date
    if end_date == None:
        end_date_dt = datetime.today()
    else:
        try:
            end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')
        except:
            raise ValueError("Please provide dates in format of YYYY-MM-DD")
            
            
    #Checking if dates make sense
    if start_date_dt > end_date_dt:
        raise ValueError("Incorrect start and end date values given!!!")
 
    #Loop in 30 day intervals adding the days, then adding strings of dates to the array
    current_date = start_date_dt
    #print(end_date_dt)
    #print(current_date)

    while current_date < end_date_dt:
        #print("waht")
        date_intervals.append(current_date.strftime('%Y-%m-%d'))
        current_date += timedelta(days=30)
        
    if current_date != end_date_dt:
        current_date = end_date_dt
        current_date += timedelta(days=-1)
        date_intervals.append(current_date.strftime('%Y-%m-%d'))
        
    return date_intervals

In [5]:
#start_date defines the date we start collecting data from, this will usually be the date that the last batch was collected
#end_today is a boolean value that signifies if we want the batch to go from the start date to the date the batch collection is called
#kwargs will be the end_today if end_today is false.
def generate_activities_json(start_date, *args, **kwargs):
    end_date = kwargs.get('end_date', datetime.today().strftime('%Y-%m-%d'))
    
    
    date_intervals = get_date_intervals(start_date, end_date)
    print(date_intervals)
    
    get_activities_url = f'https://api.fitbit.com/1/user/-/activities/list.json'
    
    for index, sd in enumerate(date_intervals, 1):

        get_activities_param = {
            #'beforeDate' : '2024-08-06',
            'afterDate' : sd,
            'sort' : 'asc',
            'limit' : 100,
            'offset' : 0
        }
        get_activities_headers = {
            "authorization" : f"Bearer {os.getenv('ACCESS_TOKEN')}"
        }

        activities_req = requests.get(url = get_activities_url, params=get_activities_param, headers=get_activities_headers)

        activities_json = activities_req.json()

        file_path = f'../data/raw_data/get_activities_{index}.json'

        with open(file_path, 'w') as json_file:
            json.dump(activities_json, json_file, indent=4)

In [6]:
generate_activities_json(start_date)

['2023-11-23', '2023-12-23', '2024-01-22', '2024-02-21', '2024-03-22', '2024-04-21', '2024-05-21', '2024-06-20', '2024-07-20', '2024-08-19', '2024-09-08']


# Daily Activity Summary

In [7]:
# #Unfortunately, there is no mass call for this one, and the api rate limit is 150 per hour. So for now I will forgoe this 
# #since most of the info in this is in the other API calls.


# get_daily_activities_url = f'https://api.fitbit.com/1/user/-/activities/date/2024-07-28.json'

# get_daily_activities_param = {
#     #'beforeDate' : '2024-07-17',
#     'date' : '2024-07-18',
#     #'sort' : 'asc',
#     #'limit' : 100,
#     #'offset' : 0
# }
# get_daily_activities_headers = {
#     "authorization" : f"Bearer {os.getenv('ACCESS_TOKEN')}"
# }

# #daily_activities_req = requests.get(url = get_daily_activities_url, params=get_daily_activities_param, headers=get_daily_activities_headers)
# daily_activities_req = requests.get(url = get_daily_activities_url, headers=get_daily_activities_headers)

# daily_activities_json = daily_activities_req.json()

# print(daily_activities_json)

In [8]:
#pprint(daily_activities_json) 

# Get Sleep Activity

In [9]:
def generate_sleep_json(start_date, *args, **kwargs):
    end_date = kwargs.get('end_date', datetime.today().strftime('%Y-%m-%d'))  
    
    date_intervals = get_date_intervals(start_date, end_date)
    print(date_intervals)
    
    get_sleep_url = f'https://api.fitbit.com/1.2/user/-/sleep/list.json'
    
    get_sleep_headers = {
            "authorization" : f"Bearer {os.getenv('ACCESS_TOKEN')}"
        }
    

    for index, sd in enumerate(date_intervals, 1):

        get_sleep_param = {
            'afterDate' : sd,
            #'beforeDate' : '2024-08-06',
            'sort' : 'asc',
            'limit' : 100,
            'offset' : 0
        }

        daily_sleep_req = requests.get(url = get_sleep_url, params=get_sleep_param, headers=get_sleep_headers)

        daily_sleep_json = daily_sleep_req.json()

        file_path = f'../data/raw_data/get_sleep_{index}.json'

        with open(file_path, 'w') as json_file:
            json.dump(daily_sleep_json, json_file, indent=4)

In [10]:
generate_sleep_json(start_date)

['2023-11-23', '2023-12-23', '2024-01-22', '2024-02-21', '2024-03-22', '2024-04-21', '2024-05-21', '2024-06-20', '2024-07-20', '2024-08-19', '2024-09-08']


# Get Skin Temperature

In [11]:
get_skin_temp_headers = {
    "authorization" : f"Bearer {os.getenv('ACCESS_TOKEN')}"
}

#cfs_req = requests.get(url = get_cfs_url, params=get_cfs_param, headers=get_cfs_headers)

dt_iter_obj = datetime.strptime(start_date, '%Y-%m-%d')
skin_json_array = []
while(dt_iter_obj < datetime.now()):
    
    end_date_str = date_plus_thirty(dt_iter_obj.strftime('%Y-%m-%d'))
    
    get_skin_temp_url = f"https://api.fitbit.com/1/user/-/temp/skin/date/{dt_iter_obj.strftime('%Y-%m-%d')}/{end_date_str}.json"
    
    skin_temp_req = requests.get(url = get_skin_temp_url, headers=get_skin_temp_headers)

    skin_temp_json = skin_temp_req.json()
    skin_json_array.append(skin_temp_json)
    
    dt_iter_obj = datetime.strptime(end_date_str, '%Y-%m-%d')




In [12]:
for index, skin_json in enumerate(skin_json_array):
    file_path = f'../data/raw_data/get_skin_{index+1}.json'
    
    
    with open(file_path, 'w') as json_file:
        json.dump(skin_json, json_file, indent=4)

In [13]:
#dateTime, date of temp skin recorded
#value : nightlyRelative: + or - value in fahrenheit from baseline temperature from up to 30 days. so +0.4 means that night 
#i was 0.4 def above my baseline.

# Heart Rate Variability

In [14]:
get_hrv_headers = {
    "authorization" : f"Bearer {os.getenv('ACCESS_TOKEN')}"
}

dt_iter_obj = datetime.strptime(start_date, '%Y-%m-%d')
hrv_json_array = []
while(dt_iter_obj < datetime.now()):
    
    end_date_str = date_plus_thirty(dt_iter_obj.strftime('%Y-%m-%d'))
    get_hrv_url = f"https://api.fitbit.com/1/user/-/hrv/date/{dt_iter_obj.strftime('%Y-%m-%d')}/{end_date_str}.json"

    hrv_req = requests.get(url = get_hrv_url, headers=get_hrv_headers)

    hrv_json = hrv_req.json()
    hrv_json_array.append(hrv_json)
    dt_iter_obj = datetime.strptime(end_date_str, '%Y-%m-%d')



In [15]:
for index, hrv_json in enumerate(hrv_json_array):
    file_path = f'../data/raw_data/get_hrv_{index+1}.json'
    
    
    with open(file_path, 'w') as json_file:
        json.dump(hrv_json, json_file, indent=4)

In [16]:
# During sleep, measure of heart rate variability 
# dailyRmssd Root Mean Square of Successive Differences between heart beats, measuring short-term variability of users daily HR.
# deepRmssd Root Mean Square of Successive Differences between heart beats, measuring short-term variability of users deep HR.

# Breathing Rate Summary

In [17]:
get_br_headers = {
    "authorization" : f"Bearer {os.getenv('ACCESS_TOKEN')}"
}

dt_iter_obj = datetime.strptime(start_date, '%Y-%m-%d')
br_json_array = []
while(dt_iter_obj < datetime.now()):
    
    end_date_str = date_plus_thirty(dt_iter_obj.strftime('%Y-%m-%d'))
    get_br_url = f"https://api.fitbit.com/1/user/-/br/date/{dt_iter_obj.strftime('%Y-%m-%d')}/{end_date_str}.json"

    br_req = requests.get(url = get_br_url, headers=get_br_headers)

    br_json = br_req.json()
    br_json_array.append(br_json)
    dt_iter_obj = datetime.strptime(end_date_str, '%Y-%m-%d')



In [18]:
for index, br_json in enumerate(br_json_array):
    file_path = f'../data/raw_data/get_br_{index+1}.json'
    
    
    with open(file_path, 'w') as json_file:
        json.dump(br_json, json_file, indent=4)

In [19]:
# breathingRate is the average number of breaths taken per minute.